In [2]:
# Import required libraries
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
# Read in the insurance dataset
df_train = pd.read_csv("/content/drive/MyDrive/Kidney_stone/train.csv")

In [8]:
 df_test = pd.read_csv("/content/drive/MyDrive/Kidney_stone/test.csv")
 df_test

,id,gravity,ph,osmo,cond,urea,calc
0,414,1.017,5.24,345,11.5,152,1.16
1,415,1.020,5.68,874,29.0,385,3.46
2,416,1.024,5.36,698,19.5,354,13.00
3,417,1.020,5.33,668,25.3,252,3.46
4,418,1.011,5.87,567,29.0,457,2.36
...,...,...,...,...,...,...,...
271,685,1.029,6.27,853,21.4,364,7.31
272,686,1.012,5.62,410,14.0,195,1.45
273,687,1.031,5.68,874,29.0,380,4.49
274,688,1.019,5.47,543,21.4,170,1.64


In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414 entries, 0 to 413
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       414 non-null    int64  
 1   gravity  414 non-null    float64
 2   ph       414 non-null    float64
 3   osmo     414 non-null    int64  
 4   cond     414 non-null    float64
 5   urea     414 non-null    int64  
 6   calc     414 non-null    float64
 7   target   414 non-null    int64  
dtypes: float64(4), int64(4)
memory usage: 26.0 KB


In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler

# Load the data
train_df = pd.read_csv("/content/drive/MyDrive/Kidney_stone/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Kidney_stone/test.csv")

# Split the features and target
X_train = train_df.drop(['id', 'target'], axis=1).values
y_train = train_df['target'].values

# Scale the features using MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

In [11]:
# Define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Implement stratified k-fold cross-validation to prevent data leakage
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train, y_train)):
    print(f'Fold {fold+1}')

    # Split the training and validation data for this fold
    X_train_fold = X_train[train_idx]
    y_train_fold = y_train[train_idx]
    X_val_fold = X_train[val_idx]
    y_val_fold = y_train[val_idx]

    # Train the model on this fold
    history = model.fit(X_train_fold, y_train_fold, epochs=500, batch_size=32, validation_data=(X_val_fold, y_val_fold))


Streaming output truncated to the last 5000 lines.
11/11 [==============================] - 0s 17ms/step - loss: 0.6638 - accuracy: 0.7039 - val_loss: 0.6728 - val_accuracy: 0.6024
Epoch 4/500
11/11 [==============================] - 0s 11ms/step - loss: 0.6465 - accuracy: 0.7009 - val_loss: 0.6626 - val_accuracy: 0.6627
Epoch 5/500
11/11 [==============================] - 0s 6ms/step - loss: 0.6287 - accuracy: 0.7130 - val_loss: 0.6548 - val_accuracy: 0.6506
Epoch 6/500
11/11 [==============================] - 0s 5ms/step - loss: 0.6100 - accuracy: 0.7190 - val_loss: 0.6478 - val_accuracy: 0.6747
Epoch 7/500
11/11 [==============================] - 0s 6ms/step - loss: 0.5985 - accuracy: 0.7281 - val_loss: 0.6440 - val_accuracy: 0.6867
Epoch 8/500
11/11 [==============================] - 0s 7ms/step - loss: 0.5862 - accuracy: 0.7009 - val_loss: 0.6422 - val_accuracy: 0.6506
Epoch 9/500
11/11 [==============================] - 0s 5ms/step - loss: 0.5729 - accuracy: 0.7130 - val_loss: 0.

9/9 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


[0.0, 0.0]

In [15]:
# Evaluate the model on the test set
X_test = test_df.drop('id', axis=1).values
X_test = scaler.transform(X_test)
y_pred = model.predict(X_test)

# Flatten the y_pred array to ensure it has shape (n_samples,)
y_pred = np.squeeze(y_pred)

# Round the predictions to 0 or 1
y_pred = np.round(y_pred).astype(int)

# Save the predictions to a CSV file
submission_df = pd.DataFrame({'id': test_df['id'], 'target': y_pred})
submission_df.to_csv('submission.csv', index=False)


9/9 [==============================] - 0s 6ms/step
